<a href="https://colab.research.google.com/github/cras-lab/BlockChain_API/blob/main/Ethereum_CoreAPI_Nodejs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

이더리움 로컬에 접속하는 예제<BR>
먼저 web3 모듈을 설치해야 한다.

In [ ]:
pip -q install web3

필요한 모듈을 임포트 한다.

In [3]:
from web3 import Web3
import matplotlib.pyplot as plt

Colab 설치시 로컬접근을 위해서는 터널링이 필요하다.<BR>
CMD창에서 "lt --port 8545 --local-host 0.0.0.0"을 실행해야 한다. <BR>
Node.js를 사용해서 터널링을 한다.node.js를 실행하고 나서 나타나는 URL을 복사해서 입력

In [4]:
# node.js를 통한 외부 접근 URL. Colab 설치시 필요
nodejs_url =  "https://thick-suits-stay.loca.lt"

블록을 읽어 올 범위 등을 설정하고, 여러 변수를 초기화 한다.

In [5]:
# 확인 범위
start_block = 1000
end_block = 1050

# 트랜잭션 개수와 평균 가스 비용을 저장할 리스트
tx_counts = []
total_fees = []
block_numbers = []

로컬 이더리움에 접속한다.

In [8]:
web3 = Web3(Web3.HTTPProvider(nodejs_url))

현재 동기화 상태인지, 초기 동기화가 끝났는지 확인한다.

In [ ]:
# 동기화 상태 확인
sync_status = web3.eth.syncing

In [ ]:
# 연결 상태 확인
if web3.is_connected() and not sync_status:     # 연결 및 동기화 완료
    print("Connected to Ethereum network...")

    # 각 블록의 트랜잭션 개수와 총 수수료 계산
    for block_number in range(start_block, end_block + 1):
        block = web3.eth.get_block(block_number, full_transactions=True)
        tx_count = len(block['transactions'])
        tx_counts.append(tx_count)
        block_numbers.append(block_number)

        if tx_count > 0:
            total_gas_fee = 0
            for tx in block['transactions']:
                receipt = web3.eth.get_transaction_receipt(tx['hash'])
                gas_used = receipt['gasUsed']
                total_gas_fee += tx['gasPrice'] * gas_used

            total_fees.append(total_gas_fee / 1e18)  # ETH 단위로 변환
        else:
            total_fees.append(0)

        print(f"Block {block_number} has {tx_count} transactions with total fee {total_gas_fee / 1e18} ETH")

    # 총 트랜잭션 개수와 평균 계산
    total_tx_count = sum(tx_counts)
    overall_average_fee = sum(total_fees) / len(total_fees)
    print(f"\nTotal transactions from block {start_block} to {end_block}: {total_tx_count}")
    print(f"Overall average fee: {overall_average_fee:.6f} ETH")

    # 그래프 생성
    fig, ax1 = plt.subplots(figsize=(10, 6))

    # 평균 수수료를 왼쪽 축에 표시
    ax1.set_xlabel('Block Number')
    ax1.set_ylabel('Average Fee (ETH)', color='blue')
    ax1.plot(block_numbers, total_fees, marker='o', linestyle='-', color='blue', label='Average Fee')
    ax1.tick_params(axis='y', labelcolor='blue')
    ax1.grid(True)

    # 평균 트랜잭션 수를 오른쪽 축에 표시
    ax2 = ax1.twinx()
    ax2.set_ylabel('Number of Transactions', color='green')
    ax2.plot(block_numbers, tx_counts, marker='o', linestyle='-', color='green', label='Number of Transactions')
    ax2.tick_params(axis='y', labelcolor='green')

    # 제목 및 범례 추가
    plt.title('Average Fee and Number of Transactions per Block')
    fig.tight_layout()
    plt.show()
elif sync_status:       # 연결되었으나 아직 동기 중
    print("Syncing in progress...")
    print(f"Starting block: {sync_status['startingBlock']}")
    print(f"Current block: {sync_status['currentBlock']}")
    print(f"Highest block: {sync_status['highestBlock']}")
    blocks_remaining = sync_status['highestBlock'] - sync_status['currentBlock']
    print(f"Blocks remaining: {blocks_remaining}")

    latest_block = web3.eth.block_number
    print(f"Latest block number: {latest_block}")

    # 네트워크 ID 조회
    network_id = web3.net.version
    print(f"Network ID: {network_id}")

    # 연결된 피어 수 조회
    peer_count = web3.net.peer_count

    print(f"Connected peers: {peer_count}")
    print("You need to wait to get access to blocks....")
else:       # 연결 안됨
    print("Failed to connect to Ethereum network via nodejs")
